## BERT

In [1]:
import pandas as pd
## for data
import json
import pandas as pd
import numpy as np
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Embedding, SpatialDropout1D
from keras.layers.recurrent import LSTM
from keras.layers import Activation, Dense
from keras.callbacks import ModelCheckpoint, EarlyStopping
## for plotting
import matplotlib.pyplot as plt
import seaborn as sns

## for processing
import re
import nltk
from nltk.tokenize import word_tokenize
## for bag-of-words
from sklearn import feature_extraction, model_selection, naive_bayes, pipeline, manifold, preprocessing, feature_selection, metrics

## for explainer
from lime import lime_text

## for word embedding
import gensim
import gensim.downloader as gensim_api

## for deep learning
from tensorflow.keras import models, layers, preprocessing as kprocessing
from tensorflow.keras import backend as K

## for bert language model
import transformers

C:\Users\Dee\anaconda3\lib\site-packages\gensim\similarities\__init__.py:15: UserWarning: The gensim.similarities.levenshtein submodule is disabled, because the optional Levenshtein package <https://pypi.org/project/python-Levenshtein/> is unavailable. Install Levenhstein (e.g. `pip install python-Levenshtein`) to suppress this warning.
  warnings.warn(msg)


In [2]:
dtf_new = pd.read_csv("data/data_hausa_cleaned.csv")

In [3]:
dtf_new = dtf_new.rename(columns={'label': 'labels', 'text_clean': 'text'})

In [4]:
dtf_new

,Unnamed: 0,news_title,labels,text
0,0,Atiku Abubakar Ya Kada Kuri'arsa A Jimeta A Ji...,Politics,atiku abubakar kada kuriarsa jimeta jihar adamawa
1,1,Kimanin mata dubu hamsin da biyar suke mutuwa ...,Health,kimanin mata dubu hamsin biyar suke mutuwa she...
2,2,Afghanistan: Masu Fashin Baki Sun Nuna Shakku ...,World,afghanistan masu fashin baki nuna shakku kan y...
3,3,Gwamnatin Kaduna Ta Gindaya Sharuddan Jinyar E...,Politics,gwamnatin kaduna gindaya sharuddan jinyar elza...
4,4,An Fara Yi Wa Jami’an Tsaron Najeriya Binciken...,Politics,an fara wa jamian tsaron najeriya binciken kwa...
...,...,...,...,...
2912,285,Yadda Karin Albashi Ke Shafar Farashin Kayayyaki,Politics,yadda karin albashi ke shafar farashin kayayyaki
2913,286,Ambaliyar Ruwan Tsunami Ta yi Barna A Indonesia,World,ambaliyar ruwan tsunami barna indonesia
2914,287,Nigeriya Zata Sami Dala Biliyan Daya Daga Maga...,World,nigeriya zata sami dala biliyan daya daga maga...
2915,288,Gwamnatin Kamaru Ta Tsare Sojoji 7 Da Ake Zarg...,World,gwamnatin kamaru tsare sojoji ake zargi kashe ...


In [5]:
data = dtf_new.drop(["Unnamed: 0", "news_title"], axis=1)

In [6]:
data

,labels,text
0,Politics,atiku abubakar kada kuriarsa jimeta jihar adamawa
1,Health,kimanin mata dubu hamsin biyar suke mutuwa she...
2,World,afghanistan masu fashin baki nuna shakku kan y...
3,Politics,gwamnatin kaduna gindaya sharuddan jinyar elza...
4,Politics,an fara wa jamian tsaron najeriya binciken kwa...
...,...,...
2912,Politics,yadda karin albashi ke shafar farashin kayayyaki
2913,World,ambaliyar ruwan tsunami barna indonesia
2914,World,nigeriya zata sami dala biliyan daya daga maga...
2915,World,gwamnatin kamaru tsare sojoji ake zargi kashe ...


In [7]:
from sklearn.preprocessing import LabelEncoder

lb = LabelEncoder()
data["labels"] = lb.fit_transform(dtf_new["labels"])

In [8]:
data

,labels,text
0,1,atiku abubakar kada kuriarsa jimeta jihar adamawa
1,0,kimanin mata dubu hamsin biyar suke mutuwa she...
2,2,afghanistan masu fashin baki nuna shakku kan y...
3,1,gwamnatin kaduna gindaya sharuddan jinyar elza...
4,1,an fara wa jamian tsaron najeriya binciken kwa...
...,...,...
2912,1,yadda karin albashi ke shafar farashin kayayyaki
2913,2,ambaliyar ruwan tsunami barna indonesia
2914,2,nigeriya zata sami dala biliyan daya daga maga...
2915,2,gwamnatin kamaru tsare sojoji ake zargi kashe ...


In [9]:
lb.classes_

array(['Health', 'Politics', 'World'], dtype=object)

In [10]:
data['labels']

0       1
1       0
2       2
3       1
4       1
       ..
2912    1
2913    2
2914    2
2915    2
2916    1
Name: labels, Length: 2917, dtype: int32

In [11]:
X = data

In [12]:
X_train, X_test = model_selection.train_test_split(X, test_size = 0.40, random_state = 42)
print(X_train.shape)
print(X_test.shape)

(1750, 2)
(1167, 2)


In [13]:
import pandas as pd

from simpletransformers.classification import ClassificationModel

# Train and Evaluation data needs to be in a Pandas Dataframe containing at least two columns. 
#If the Dataframe has a header, it should contain a 'text' and a 'labels' column. 
#If no header is present, the Dataframe should contain at least two columns, 
#with the first column is the text with type str, and the second column in the label with type int.
train_data = [
    ["Example sentence belonging to class 1", 1],
    ["Example sentence belonging to class 0", 0],
    ["Example eval senntence belonging to class 2", 2],
]
train_df = pd.DataFrame(train_data)

eval_data = [
    ["Example eval sentence belonging to class 1", 1],
    ["Example eval sentence belonging to class 0", 0],
    ["Example eval senntence belonging to class 2", 2],
]
eval_df = pd.DataFrame(eval_data)

# Create a ClassificationModel
model = ClassificationModel(
    "bert",
    "bert-base-cased",
    num_labels=5,
    args={"reprocess_input_data": True, "overwrite_output_dir": True},
    use_cuda=False
    
)

# Train the model
model.train_model(X_train)

# Evaluate the model
result, model_outputs, wrong_predictions = model.eval_model(X_test)

predictions, raw_outputs = model.predict(["Some arbitary sentence"])

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at b

  0%|          | 0/1750 [00:00<?, ?it/s]

Epoch:   0%|          | 0/1 [00:00<?, ?it/s]

Running Epoch 0 of 1:   0%|          | 0/219 [00:00<?, ?it/s]

  0%|          | 0/1167 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/146 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

In [14]:
import torch
torch.save(model, "model/bert_model_hausa")

In [14]:
import torch
model = torch.load('model/bert_model_hausa')

In [15]:
predictions, raw_outputs = model.predict(["Some arbitary sentence"])

In [17]:
result, model_outputs, wrong_predictions = model.eval_model(X_test)

In [29]:
a = list(X_test['text'])
predictions, raw_outputs = model.predict(a)

In [30]:
predictions

array([2, 1, 0, ..., 1, 2, 1], dtype=int64)

In [34]:
from sklearn.metrics import classification_report

print(classification_report(np.array(X_test["labels"]), predictions))

              precision    recall  f1-score   support

           0       0.89      0.78      0.83       260
           1       0.81      0.79      0.80       421
           2       0.79      0.86      0.82       486

    accuracy                           0.82      1167
   macro avg       0.83      0.81      0.82      1167
weighted avg       0.82      0.82      0.82      1167

